In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import torch.optim as optim 

In [14]:

# Hype-parameters
num_epochs = 4 # how many times we are running 
batch_size = 32 # 
learning_rate = .001 # 

mnist = datasets.MNIST(
    root='./data', # where to store data
    train=True, # tell the code it is training data
    download=True, # download the data
    transform=transforms.ToTensor() # transform dataset to tensor directly (no preprocessing)
) # import data
print(len(mnist))

mnist_train, mnist_test = torch.utils.data.random_split(mnist, [0.8, .2]) # split data 80/20
print(len(mnist_train))
print(len(mnist_test))  


60000
48000
12000


In [15]:
batch_size = 32

# init dataloaders to load batches into model
train_dl  = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True  ) 

test_dl   = torch.utils.data.DataLoader(mnist_test,  batch_size=10000,      shuffle=False ) 




In [16]:
# function to measure accuracy, confusion matrix, precision, recall ,f1 (metrics to measure classification)
def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.6f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted'))
    f1_measure = f1_score(y_true=y_test, y_pred=y_pred, average='weighted')
    print('F1-mesure: %.3f' % f1_measure)
    return f1_measure

In [17]:
class Classifier_CNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.model = nn.Sequential(
            ## Convolitional Layer 1
                nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1), # 1 input 16 filters, padding so same dim
                nn.ReLU(), # ReLU introduce non-linearity
                nn.MaxPool2d(2, 2), #pool
 
                ## Convolutional Layer 2
                nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1), # 16 inputs 32 filters
                nn.ReLU(),
                nn.MaxPool2d(2, 2),   
 
                ## feed forward layer w/ 1024 neurons, regular layer
                nn.Flatten(),
                nn.Linear(800, 1024),    ## see how to get 800 below on last cell
                nn.ReLU(),

                nn.Linear(1024, 10), # maps to output w/ 10 classes
                nn.LogSoftmax(dim=1)
        )
   
    def forward(self, inputs):
            
        return self.model(inputs)
        
def training_loop( num_epochs, model, loss_fn, opt):

    losses_list = []
    
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            
            ## print( xb.shape )   ## check this comes out as [N, 1, 28, 28]
            ## yb = torch.squeeze(yb, dim=1)
            
            y_pred = model(xb)
            loss   = loss_fn(y_pred, yb)
    
            opt.zero_grad()
            loss.backward()
            opt.step()
            
        if epoch % 1 == 0:
            print(epoch, "loss=", loss)
            losses_list.append(  loss  )
            
    return losses_list



In [18]:
model = Classifier_CNN() # create our model

opt = torch.optim.Adam(model.parameters(), lr = learning_rate) # optimizer that does steps

loss_fn = nn.CrossEntropyLoss() # type of loss func

my_losses_list = training_loop(num_epochs, model, loss_fn, opt)

0 loss= tensor(0.0142, grad_fn=<NllLossBackward0>)
1 loss= tensor(0.0030, grad_fn=<NllLossBackward0>)
2 loss= tensor(0.1251, grad_fn=<NllLossBackward0>)
3 loss= tensor(0.0001, grad_fn=<NllLossBackward0>)


In [19]:

with torch.no_grad(): # detach grad tracking for tensors
    for x_real, y_real in test_dl:
        
        y_pred = model(  x_real  )
        
        vals, indeces = torch.max( y_pred, dim=1  )
        preds = indeces
        print_metrics_function(y_real, preds)

Accuracy: 0.987900
Confusion Matrix:
[[ 963    0    0    0    0    1    0    0    0    2]
 [   0 1126    1    0    0    0    1    2    0    0]
 [   2    1  989    1    0    0    0   11    0    0]
 [   0    0    6  997    0    6    0    9    3    0]
 [   0    1    1    0  969    0    1    1    1    0]
 [   1    0    0    0    0  889    0    0    1    0]
 [   2    0    1    0    4   14  952    0    2    0]
 [   0    4    0    0    1    0    0 1081    2    1]
 [   1    2    3    1    2    1    0    3  929    0]
 [   0    2    0    0    8    4    0    9    1  984]]
Precision: 0.988
Recall: 0.988
F1-mesure: 0.988
Accuracy: 0.989000
Confusion Matrix:
[[181   0   0   0   0   0   0   0   0   0]
 [  0 200   0   0   0   0   0   1   0   0]
 [  1   0 199   0   0   0   0   2   1   0]
 [  0   0   0 194   0   2   0   0   0   1]
 [  0   0   0   0 202   0   0   0   0   1]
 [  0   0   0   0   0 187   1   0   1   0]
 [  0   0   0   0   0   3 188   0   1   0]
 [  0   1   0   0   0   0   0 222   0   0]
 [ 

In [ ]:
# k-fold cross-validation
from sklearn.model_selection import cross_val_score

# Example: k-fold cross-validation with a model
#cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

In [20]:
# commented out, took out validation idk if we need

#from torch.utils.data import DataLoader
#validation_loader = DataLoader(mnist_valid, batch_size=64, shuffle=False)

# Iterate through the DataLoader to validate
#for X_valid, y_valid in validation_loader:
    # Use X_valid and y_valid for validation here
    #break  # Remove this if processing the entire validation set
#y_valid_pred = model(X_valid)  # Predictions on validation set

#val_accuracy = accuracy_score(y_valid, y_valid_pred)
#print(f"Validation Accuracy: {val_accuracy:.2f}")